In [1]:
import sys
sys.path

['D:\\Anaconda\\envs\\position_completion_new\\python36.zip',
 'D:\\Anaconda\\envs\\position_completion_new\\DLLs',
 'D:\\Anaconda\\envs\\position_completion_new\\lib',
 'D:\\Anaconda\\envs\\position_completion_new',
 '',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\win32',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\win32\\lib',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\Pythonwin',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\93973\\.ipython']

In [2]:
print(sys.version)
print(sys.executable)

3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
D:\Anaconda\envs\position_completion_new\python.exe


In [3]:
import pandas as pd
import numpy as np
from collections import Iterable
import datetime
import time
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

In [4]:
import os
os.chdir("../")

#### 1.数据读取

In [5]:
# 文件名称简介见’文件名称字典.txt‘
df_fund_portfolio = pd.read_csv('./Data/ChinaMutualFundStockPortfolio.csv')
stock_value_to_nav = pd.read_csv("./Data/StkValuetToFundNAV.csv",index_col = "基金代码")
industry_value_to_nav = pd.read_csv("./Data/IndustryValueToNAV.csv")
# stock_fund_info = pd.read_csv("./Data/StkFundInfo.csv")
stk_actual_publish_date = pd.read_excel("./Data/StkActualPublishDate.xlsx",index_col = "股票代码")
fund_actual_publish_date = pd.read_excel("./Data/FundActualPublishDate.xlsx",index_col = "基金代码")
month_begin_end_df = pd.read_excel("./Data/MonthBeginEnd.xlsx")
new_skt_ind = pd.read_excel("./Data/StkIndustry.xlsx",index_col = "股票代码")
fund_type = pd.read_excel("./Data/FundType.xlsx")
pct_skttotnav = pd.read_excel('./Data/PctStkTotalNAV.xlsx',index_col = "基金代码")
fund_full_name = pd.read_excel("./Data/FundFullName.xlsx")

#修改时间格式
df_fund_portfolio.F_PRT_ENDDATE = pd.to_datetime(df_fund_portfolio.F_PRT_ENDDATE,)
df_fund_portfolio.ANN_DATE = pd.to_datetime(df_fund_portfolio.ANN_DATE)
stock_value_to_nav.columns = pd.to_datetime(stock_value_to_nav.columns)
industry_value_to_nav.F_PRT_ENDDATE = pd.to_datetime(industry_value_to_nav.F_PRT_ENDDATE)
# stock_fund_info.F_PRT_ENDDATE = pd.to_datetime(stock_fund_info.F_PRT_ENDDATE)
# stock_fund_info.ANN_DATE = pd.to_datetime(stock_fund_info.ANN_DATE)
stk_actual_publish_date.columns = pd.to_datetime(stk_actual_publish_date.columns)
fund_actual_publish_date.columns = pd.to_datetime(fund_actual_publish_date.columns)

In [6]:
df_fund_portfolio.drop(["CRNCY_CODE","F_PRT_STKQUANTITY","F_PRT_POSSTKVALUE","F_PRT_POSSTKQUANTITY","F_PRT_POSSTKTONAV",\
                        "F_PRT_PASSTKEVALUE","F_PRT_PASSTKQUANTITY","F_PRT_PASSTKTONAV","OPDATE","OPMODE","INDUSTRY_CODE","INDUSTRY_NAME"],axis = 1)

,S_INFO_WINDCODE,F_PRT_ENDDATE,S_INFO_STOCKWINDCODE,F_PRT_STKVALUE,F_PRT_STKVALUETONAV,ANN_DATE,STOCK_PER,FLOAT_SHR_PER,INDUSTRY_NUM,F_PRT_INDVALUETONAV
0,519908.OF,1998-07-31,600895.SH,2.492654e+07,1.21,1998-08-07,2.66,2.89,11.0,NaN
1,519908.OF,1998-09-30,000063.SZ,1.387409e+08,6.95,1998-10-09,14.99,6.06,3.0,NaN
2,519908.OF,1998-09-30,000069.SZ,2.309391e+07,1.15,1998-10-09,2.50,2.44,11.0,NaN
3,519908.OF,1998-09-30,000400.SZ,6.593623e+07,3.30,1998-10-09,7.12,5.49,3.0,NaN
4,519908.OF,1998-09-30,000670.SZ,2.054146e+07,1.03,1998-10-09,2.22,1.40,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...
6235808,005245.OF,2021-06-30,688575.SH,1.102291e+05,0.02,2021-08-31,0.02,0.01,3.0,77.676132
6235809,005244.OF,2021-06-30,688575.SH,1.102291e+05,0.02,2021-08-31,0.02,0.01,3.0,77.676132
6235810,005245.OF,2021-06-30,300918.SZ,1.033297e+05,0.02,2021-08-31,0.02,0.01,3.0,77.676132
6235811,005244.OF,2021-06-30,300918.SZ,1.033297e+05,0.02,2021-08-31,0.02,0.01,3.0,77.676132


#### 2.数据清洗

##### (1)去除lof基金

In [7]:
# 去除LOF基金
df_fund_portfolio = df_fund_portfolio.loc[~df_fund_portfolio.loc[:,'S_INFO_WINDCODE'].map(lambda x: x[-2:] in ["SH","SZ"])]
df_fund_portfolio.reset_index(inplace = True,drop = True)

##### (2)剔除分级基金

In [8]:
"""
集合理财计划名称相同，但是基金简称不同，只保留其中一只基金，排序后去重大概率保留A类原始基金
对集合理财计划、公告日期和股票代码相同的基金去重
"""

'\n集合理财计划名称相同，但是基金简称不同，只保留其中一只基金，排序后去重大概率保留A类原始基金\n对集合理财计划、公告日期和股票代码相同的基金去重\n'

In [9]:
df_fund_portfolio = pd.merge(left = df_fund_portfolio,right = fund_full_name,left_on = "S_INFO_WINDCODE",right_on = "S_INFO_WINDCODE") 

In [10]:
df_fund_portfolio.sort_values(by = 'FUND_SIMPLIFIED_NAME',ascending = True,inplace = True) # 使得A类基金能够排在前面

In [11]:
df_fund_portfolio.drop_duplicates(["S_INFO_STOCKWINDCODE","F_PRT_ENDDATE","FUND_FULL_NAME"],inplace = True) # 对集合理财计划、公告日期和股票代码相同的基金去重

##### (3)保留偏股混合型、普通股票型基金

In [12]:
stk_bond_fund_list = pd.read_excel("./Data/StkBondFundList.xlsx") # 偏股混合型基金
stk_fund_list = pd.read_excel("./Data/StkFundList.xlsx") # 普通股票型基金

In [13]:
# 只选择在偏股混合型基金和普通股票型基金列表中存在的基金
df_fund_portfolio.query("S_INFO_WINDCODE in {} or S_INFO_WINDCODE in {}".format(stk_bond_fund_list.S_INFO_WINDCODE.tolist(),stk_fund_list.S_INFO_WINDCODE.tolist()),inplace = True)

##### (4)公告日期对齐

In [14]:
df_fund_portfolio.sort_values(by = ["F_PRT_ENDDATE","F_PRT_STKVALUE"],ascending = True,inplace = True) # 分级排序，按照日期和股票占净值比从小到大排序

In [15]:
df_fund_portfolio.reset_index(drop = True,inplace = True)

In [16]:
"""
理论上定期报告的代理日期应该在月底，但是部分数据存在特殊情况
处理方法为:如果存在月底的报告数据，那么去除当月非月底的数据；若无则将该数据设定为定期报告数据
月份非3、6、9、12的数据不予考虑，不知道是什么东西
"""

'\n理论上定期报告的代理日期应该在月底，但是部分数据存在特殊情况\n处理方法为:如果存在月底的报告数据，那么去除当月非月底的数据；若无则将该数据设定为定期报告数据\n月份非3、6、9、12的数据不予考虑，不知道是什么东西\n'

In [17]:
# 只选择整齐在季度月底发布公告的那些信息
neat_fund_portfolio = df_fund_portfolio.loc[(df_fund_portfolio.F_PRT_ENDDATE.apply(lambda x:x.month).isin([3,6,9,12]))&\
                      (df_fund_portfolio.F_PRT_ENDDATE.apply(lambda x:x.day).isin([30,31]))]
# 虽然在季度当月，但是却不是月底的信息
dirty_fund_portfolio = df_fund_portfolio.loc[(~df_fund_portfolio.F_PRT_ENDDATE.apply(lambda x:x.month).isin([3,6,9,12]))|\
                       (~df_fund_portfolio.F_PRT_ENDDATE.apply(lambda x:x.day).isin([30,31]))]

In [18]:
# 大致有74只公告非常日期的基金，多数为LOF基金，目前先剔除
dirty_fund_portfolio.S_INFO_WINDCODE.unique()

array(['160505.OF', '162703.OF', '162605.OF', '161903.OF', '160106.OF',
       '161706.OF', '161005.OF', '162207.OF', '160607.OF', '166001.OF',
       '162006.OF', '161610.OF', '160611.OF', '163503.OF', '160916.OF',
       '160314.OF', '020018.OF', '165313.OF', '161810.OF', '160212.OF',
       '166006.OF', '163406.OF', '166007.OF', '164205.OF', '165508.OF',
       '166009.OF', '160512.OF', '165512.OF', '160415.OF', '160127.OF',
       '166011.OF', '163412.OF', '165516.OF', '163415.OF', '005632.OF',
       '160918.OF', '160133.OF', '501028.OF', '501032.OF', '160322.OF',
       '161038.OF', '501038.OF', '166024.OF', '501051.OF', '161131.OF',
       '501065.OF', '166025.OF', '161912.OF', '501063.OF', '501070.OF',
       '160645.OF', '501088.OF', '501087.OF', '161837.OF', '167508.OF',
       '501091.OF', '169107.OF', '501092.OF', '160926.OF', '506002.OF',
       '161040.OF', '506000.OF', '160325.OF', '506006.OF', '506003.OF',
       '506005.OF', '166027.OF', '506001.OF', '168207.OF', '1619

In [19]:
# 选择公告日期完全在四季度月底的基金
neat_fund_portfolio.query("S_INFO_WINDCODE not in {}".format(dirty_fund_portfolio.S_INFO_WINDCODE.unique().tolist()),inplace = True)

In [20]:
neat_fund_portfolio.reset_index(drop = True,inplace = True)

In [21]:
# 去除历史上出现跳月的基金，跳月即两个最近的公告月份大于三月，实际上这样的补全并没有意义，比如说2018-06-30->2018-12-31
interrupted_fund = neat_fund_portfolio.loc[:,["S_INFO_WINDCODE","F_PRT_ENDDATE"]]

In [22]:
interrupted_fund.sort_values(["S_INFO_WINDCODE","F_PRT_ENDDATE"],inplace = True)

In [23]:
interrupted_fund.drop_duplicates(inplace = True)

In [24]:
interrupted_fund.loc[:,"DAYS_DIFF"] = (interrupted_fund.groupby('S_INFO_WINDCODE')["F_PRT_ENDDATE"].diff()/30).astype('timedelta64[D]').astype(float)

In [25]:
interrupted_fund.fillna(3.0,inplace = True)

In [26]:
interrupted_fund_list = interrupted_fund.query("DAYS_DIFF != 3").S_INFO_WINDCODE.unique().tolist() # 若出现上一次报告不是一个季度以前，那么就没必要考虑了

In [27]:
neat_fund_portfolio.query(f"S_INFO_WINDCODE not in {interrupted_fund_list}",inplace = True) # 直接把中间出现跳月的基金去除

In [28]:
# 当然还可以考虑基金经理换届，这里就不考虑了

#### 3.补全持仓并输出持仓明细

In [29]:
neat_fund_portfolio.reset_index(drop = True,inplace = True)

start_date = "2013-12-31"
end_date = "2021-06-30"

In [30]:
for tmp_fund in neat_fund_portfolio.S_INFO_WINDCODE.unique():
    
    print(f"processing {tmp_fund}")
    
    fund_portfolio = neat_fund_portfolio.query("S_INFO_WINDCODE == '{}'".format(tmp_fund)) # 每支基金都保存一个持仓excel
    fund_portfolio.sort_values(by = ["F_PRT_ENDDATE","F_PRT_STKVALUE"],ascending = True,inplace = True) #分级排序，按照日期和股票占净值比从小到大排序
    
    # 3.6会报错
#     fund_stock_value = fund_portfolio.groupby('F_PRT_ENDDATE')[['S_INFO_STOCKWINDCODE','F_PRT_STKVALUE','STOCK_PER',\
#                                                                 'F_PRT_STKVALUETONAV','INDUSTRY_NUM','F_PRT_INDVALUETONAV']].agg(list)
    
    S_INFO_STOCKWINDCODE_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["S_INFO_STOCKWINDCODE"].apply(list)
    F_PRT_STKVALUE_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["F_PRT_STKVALUE"].apply(list)
    STOCK_PER_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["STOCK_PER"].apply(list)
    F_PRT_STKVALUETONAV_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["F_PRT_STKVALUETONAV"].apply(list)
    INDUSTRY_NUM_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["INDUSTRY_NUM"].apply(list)
    F_PRT_INDVALUETONAV_agg = fund_portfolio.groupby('F_PRT_ENDDATE')["F_PRT_INDVALUETONAV"].apply(list)
    
    fund_stock_value = pd.concat([S_INFO_STOCKWINDCODE_agg,F_PRT_STKVALUE_agg,STOCK_PER_agg,F_PRT_STKVALUETONAV_agg,INDUSTRY_NUM_agg,F_PRT_INDVALUETONAV_agg],axis = 1)
    
    # 存储所有报告实际持仓/模拟持仓的字典
    holding_info = dict()
    
    # 由于2013年以前信息披露与之后不同，因此直接从13年底开始补全
    fund_stock_value_between = fund_stock_value.loc[start_date:end_date,:].copy()
    
    stock_weight = stock_value_to_nav.loc[tmp_fund,:] # 获取当前基金的股票占基金净值比数据
    temp_industry_value_to_nav = industry_value_to_nav.dropna().query("S_INFO_WINDCODE == '{}'".format(tmp_fund)) # 获取基金的行业占净值比
    
    for date,feats in fund_stock_value_between.iterrows():
        if pd.to_datetime(date).month%6 == 0: # 假如报告发布月份为年报或者半年班，直接就是全持仓
            tmp_data = dict(zip(feats.S_INFO_STOCKWINDCODE,feats.F_PRT_STKVALUETONAV))
        else:
            if stock_weight.loc[date] - np.nansum(feats.F_PRT_STKVALUETONAV) <= 0: # 假如前10大重仓股已经满了，即成为全持仓
                tmp_data = dict(zip(feats.S_INFO_STOCKWINDCODE,feats.F_PRT_STKVALUETONAV))

            last_fund_stock_value_between = fund_stock_value.shift(1).loc[date] # 读取上季度的全持仓信息
            if last_fund_stock_value_between.dropna().empty: # 如果上季度没有全持仓信息，那么就是第一期就是一三季度，无法补全持仓
                continue

            last_stock_list = last_fund_stock_value_between.S_INFO_STOCKWINDCODE # 上季度持仓的股票列表
            present_stock_list = feats.S_INFO_STOCKWINDCODE # 本季度持仓的股票列表

            # 转换成df操作
            # 上一个季度的股票占比、所属证监会行业代码和行业占净值比
            last_stock_df = np.transpose(pd.DataFrame([last_fund_stock_value_between.F_PRT_STKVALUETONAV,\
                                                       last_fund_stock_value_between.INDUSTRY_NUM,last_fund_stock_value_between.F_PRT_INDVALUETONAV]))
            last_stock_df.index = last_stock_list
            last_stock_df.columns = ["F_PRT_STKVALUETONAV","INDUSTRY_NUM","F_PRT_INDVALUETONAV"]
            # 本季度的
            present_stock_df = np.transpose(pd.DataFrame([feats.F_PRT_STKVALUETONAV,feats.INDUSTRY_NUM,feats.F_PRT_INDVALUETONAV]))
            present_stock_df.index = present_stock_list
            present_stock_df.columns = ["F_PRT_STKVALUETONAV","INDUSTRY_NUM","F_PRT_INDVALUETONAV"]    

            # 对本季度的前十大重仓股持仓信息，按照行业求所占净值比
            present_ind_total_pct = present_stock_df.groupby(["INDUSTRY_NUM"])["F_PRT_STKVALUETONAV"].apply(np.sum)

            # 下面的数据是补充数据，记录了该基金在该季度的披露的证监会一级行业占基金净值比
            present_ind_value_to_nav = temp_industry_value_to_nav.query("F_PRT_ENDDATE == '{}'".format(date))
            if present_ind_value_to_nav.empty:
                continue
            present_ind_value_to_nav.set_index('INDUSTRY_NUM',inplace = True)
            
            # 信息披露垃圾，比如519002.OF的14年一季报，这里需要额外处理垃圾数据，假如说底层数据出现了未出现在季报中披露的行业，则删除
            index_intersection = set(present_ind_total_pct.index).intersection(set(present_ind_value_to_nav.index))
            
            # 本季度的披露减去前十大重仓股的行业净值之和，剩下的为本季度的非前十大重仓股的占基金净值比，这部分是需要模拟分配的那部分净值
            present_ind_value_to_nav.loc[index_intersection,"F_PRT_INDVALUETONAV"] = \
            present_ind_value_to_nav.loc[index_intersection,"F_PRT_INDVALUETONAV"] - present_ind_total_pct

            # 上季度持仓不在本季度重仓股的股票，净值占比为上季度的
            left_stock_list = set(last_stock_list) - set(present_stock_list)
            left_stock_df = last_stock_df.loc[left_stock_list,:]

            # 若本季度不再配置某行业，去除上个季度对于该行业的配置
            present_ind_num = present_ind_value_to_nav.index.tolist()
            left_stock_df = left_stock_df.query("INDUSTRY_NUM in {}".format(present_ind_num))

            # 合并两表的准备工作
            left_stock_df.loc[:,"S_INFO_STOCKWINDCODE"] = left_stock_df.index
            present_ind_value_to_nav.loc[:,"INDUSTRY_NUM"] = present_ind_value_to_nav.index
            present_ind_value_to_nav.reset_index(inplace = True,drop = True)

            # 去除两个df中重复的列，防止带入后缀
            present_ind_value_to_nav.drop(labels = ["S_INFO_WINDCODE","F_PRT_ENDDATE"],axis = 1,inplace = True)
            present_ind_value_to_nav = present_ind_value_to_nav[present_ind_value_to_nav.F_PRT_INDVALUETONAV>0]
            left_stock_df.drop(labels = ["F_PRT_INDVALUETONAV"],axis = 1,inplace = True)

            # 按照上一期的行业将本期的剩余净值分配出去
            left_stock_df = pd.merge(left = left_stock_df,right = present_ind_value_to_nav,\
                            left_on = "INDUSTRY_NUM",right_on = "INDUSTRY_NUM",how = "left")

            left_ind_total_pct = left_stock_df.groupby(["INDUSTRY_NUM"])[["F_PRT_STKVALUETONAV"]].sum()
            left_ind_total_pct.columns = ["UNALLOCATE_INDNAV"]
            left_ind_total_pct.loc[:,"INDUSTRY_NUM"] = left_ind_total_pct.index
            left_ind_total_pct.reset_index(inplace = True,drop = True)

            left_stock_df = pd.merge(left = left_stock_df,right = left_ind_total_pct,\
                            left_on = "INDUSTRY_NUM",right_on = "INDUSTRY_NUM",how = "left")
            #按照上季度每只股票占行业剩余净值之比，将本季度所剩的净值比例分配
            stknav_to_indnav_pre = left_stock_df.F_PRT_STKVALUETONAV*left_stock_df.F_PRT_INDVALUETONAV/left_stock_df.UNALLOCATE_INDNAV  

            # 本期前十大重仓股和模拟出来的重仓股合并，得出本季度全持仓模拟信息
            tmp_data = dict(zip(left_stock_df.S_INFO_STOCKWINDCODE,stknav_to_indnav_pre.astype(float)))
            tmp_data.update(dict(zip(present_stock_df.index,present_stock_df.F_PRT_STKVALUETONAV.astype(float))))

        holding_info[date] = tmp_data
    
    # 存储持仓信息的表格
    holding_df = pd.DataFrame()
    
    for date,holdings in holding_info.items(): # 行为每季度的月底，列为所涉及的全部股票
        tmp_df = pd.DataFrame(index = [date],columns = holdings.keys())
        tmp_df.loc[date,:] = list(holdings.values())
        holding_df = holding_df.append(tmp_df)
    
    # 剩余的仓位用中证全债替代
    holding_df.loc[:,"H11001.CSI"] = 100 - holding_df.sum(axis = 1)
    
    # 保存持仓信息
#     holding_df.to_excel("./Output/PositionCompletionDetail/{}.xlsx".format(tmp_fund[:6]))

processing 519002.OF
processing 519019.OF
processing 050008.OF
processing 519606.OF
processing 519915.OF
processing 160919.OF
processing 050009.OF
processing 202005.OF
processing 202007.OF
processing 519035.OF
processing 110013.OF
processing 110029.OF
processing 519039.OF
processing 213008.OF
processing 519017.OF
processing 040008.OF
processing 040007.OF
processing 121008.OF
processing 161601.OF
processing 080001.OF
processing 150968.OF
processing 070001.OF
processing 217001.OF
processing 240001.OF
processing 070002.OF
processing 070003.OF


KeyboardInterrupt: 